In [1]:
from pyflink.common import WatermarkStrategy, Row
from pyflink.common.serialization import Encoder
from pyflink.common.typeinfo import Types
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.datastream.connectors import FileSink, OutputFileConfig, NumberSequenceSource
from pyflink.datastream.functions import RuntimeContext, MapFunction
from pyflink.datastream.state import ValueStateDescriptor
from pyflink.common.serialization import JsonRowDeserializationSchema
from pyflink.common.typeinfo import Types
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.datastream.connectors import FlinkKafkaConsumer,FlinkKafkaProducer
from pyflink.common.serialization import SimpleStringSchema, SerializationSchema,JsonRowSerializationSchema,Encoder
from pyflink.common.typeinfo import Types
import json
from pyflink.common import Configuration

In [2]:
# https://nightlies.apache.org/flink/flink-docs-master/docs/dev/python/python_config/
# wget  -P $FLINK_HOME/lib https://repo1.maven.org/maven2/org/apache/flink/flink-sql-connector-kafka/1.15.3/flink-sql-connector-kafka-1.15.3.jar

#     kafka-topics  --create --bootstrap-server broker:9092 --replication-factor 1 --partitions 3 --topic sensor-data
#     kafka-console-producer --bootstrap-server broker:9092 --topic sensor-data

#  $FLINK_HOME/bin/flink  run -m localhost:8181 -py /home/training/fastdata-stack/flink/code/KafkaHelloWorld.py
#  $FLINK_HOME/bin/flink  run -m localhost:8282 -py /home/training/fastdata-stack/flink/code/KafkaHelloWorld.py

# {"tag": "sensor1", "value": 32.3}
# {"tag": "sensor2", "value": 34.3}
#config = Configuration()
#config.set_integer("python.fn-execution.bundle.size", 1000)

#python.executable


In [3]:

env = StreamExecutionEnvironment.get_execution_environment()
env.enable_checkpointing(1000 * 10) # 10 sec
# use case 1 : App crashes, restart in another container/task manager, this point is used to recover data
# use case 2: /application stopped, restarted by developer to continue from where it left
env.get_checkpoint_config().set_checkpoint_storage_dir("hdfs://namenode:9000/checkpoints-data-jupyter")


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/flink-1.15.3/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop-2.7.7/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


In [ ]:
# save points
# is snapshots wokrs very similar to checkpoints, the state is stored into DFS, with metadata, directory path etc
# save point must point to unique directory , NOT two save points to same directory

# you plan an update rollout, system maintenance, rollback feature, like versioning : Jan 02 2023
# you take snapshot using save point , stored in teh directory hdfs://savepoints/appname1/jan-02-2023 - this has good state until jan 02 2023 before new deployment..
# you deployed new released, something messed up

# on Jan 04, 2023 someone notice issues, here we consumed, calculated results, updated targets etc...

# you fix the code on Jan 05, 2023, deploy the new code [check is still very latest until Jan 05, we cannot use that check point, as it has incorrect state]
# Here we can do save point
# stop your application, deploy new application.. then resume the application from saved save point hdfs://savepoints/appname1/jan-02-2023



In [4]:

deserialization_schema = JsonRowDeserializationSchema.builder().type_info(
                             type_info=Types.ROW_NAMED(
                             ["tag","value"], [Types.STRING(), Types.DOUBLE()])).build()


In [5]:

kafka_consumer = FlinkKafkaConsumer(
    topics='sensor-data',
    deserialization_schema=deserialization_schema,
    properties={'bootstrap.servers': 'broker:9092', 'group.id': 'sensor-data-group-1'})


In [6]:
# create a data stream from kafka consmer 
ds = env.add_source(kafka_consumer)

In [7]:
ds.print()

In [ ]:
env.execute('state_access_demo')

6> +I[sensor1,32.3]
5> +I[sensor2,34.3]
4> +I[sensor1,32.3]
4> +I[sensor2,34.3]
